In [2]:
import pandas as pd
import numpy as np

#### Cleaning the Censo dataset

This dataset contains information on the average price of rent in different neighborhoods of Barcelona

In [3]:
# Estimacion del precio mensual de locales commerciales por barrio en años 2011, 2010, 2009, 2008
df1 = pd.read_csv('../Datos CSV/locveevolucio.csv')
relabel = {'BARRIS': 'NOM', 'DOSZEROONZE': '2011', 'DOSZERODEU': '2010', 'DOSZEROZERONOU': '2009', 'DOSZEROZEROVUIT': '2008'}
df1.rename(columns=relabel, inplace=True)
df1.drop(columns='DTE', inplace=True)

In [4]:
# Function to clean the data
def replace_characters(df, columns):
    for column in columns:
        df[column] = df[column].str.replace('[\.,]', '')
        df[column] = df[column].str.strip()
    return df

def remove_numbers(df, columns):
    for column in columns:
        df[column] = df[column].str.replace('\d', '')
        df[column] = df[column].str.strip()
    return df

In [6]:
# Cleaning the data, applying the functions
replace_characters(df1, ['NOM', '2011', '2010', '2009', '2008'])
remove_numbers(df1, ['NOM'])

/var/folders/hp/ytf1kqk92hd11vlmtk9zz38r0000gn/T/ipykernel_50542/1137789237.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace('[\.,]', '')
/var/folders/hp/ytf1kqk92hd11vlmtk9zz38r0000gn/T/ipykernel_50542/1137789237.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace('\d', '')


,NOM,2011,2010,2009,2008
0,el Raval,2176,2445,2773,3065
1,el Barri Gòtic,2632,3026,3750,3828
2,la Barceloneta,2135,3005,--,--
3,Sant Pere Santa Caterina i la Ribera,2482,2828,3146,3450
4,el Fort Pienc,2224,2422,2736,2893
...,...,...,...,...,...
68,Diagonal Mar i el Front Marítim del Poblenou,2651,2991,2445,3752
69,el Besòs i el Maresme,2121,2751,2842,3669
70,Provençals del Poblenou,2255,2516,2660,3857
71,Sant Martí de Provençals,1812,2175,2266,1957


In [7]:
df1 = df1[~(df1 == '--').any(axis=1)]
# Correct data types
df1[['2011', '2010', '2009', '2008']] = df1[['2011', '2010', '2009', '2008']].astype(int)

/Users/joaquinortega/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [8]:
df1['price level'] = df1['2011'] - df1['2011'].mean()

/var/folders/hp/ytf1kqk92hd11vlmtk9zz38r0000gn/T/ipykernel_50542/2974047653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['price level'] = df1['2011'] - df1['2011'].mean()


In [11]:
df1.head(3)

,NOM,2011,2010,2009,2008,price level
0,el Raval,2176,2445,2773,3065,157.226415
1,el Barri Gòtic,2632,3026,3750,3828,613.226415
3,Sant Pere Santa Caterina i la Ribera,2482,2828,3146,3450,463.226415


In [9]:
# df1.to_csv('../Datos CSV/precios_clean.csv', index=False)

### Censo Comercial Dataset

In [12]:
df2 = pd.read_csv('../Datos CSV/2019_censcomercialbcn_detall.csv')

/Users/joaquinortega/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (35,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
# direccion = 'Codi_Via', 'Nom_Via', 'Planta', 'Porta','Lletra_Inicial'
censo_cols = ['ID_Bcn_2019','Nom_Principal_Activitat','Nom_Activitat','Nom_Sector_Activitat','Nom_Grup_Activitat','Nom_Local','Latitud','Longitud','Nom_Districte', 'Nom_Barri']
censo_clean = pd.DataFrame(df2[censo_cols])
censo_clean.rename(columns={'Nom_Principal_Activitat':'Actividad Economica','Nom_Activitat':'Nombre de Actividad', 'Nom_Barri': 'Nombre del Barrio'}, inplace=True)
censo_clean['Actividad Economica'].replace({'Actiu': 'For Profit', 'Sense activitat Econòmica': 'Non Profit'}, inplace=True)

In [14]:
first_lambda = ['Sense informació','altres','Altres ( per exemple VENDING)', 'Altres']

In [15]:
# Cambiar con la tercera columna - Nom_Grup_Activitat = First Lambda
censo_clean['Nombre de Actividad'] = censo_clean.apply(lambda row: row['Nom_Grup_Activitat'] if row['Nombre de Actividad'] in first_lambda else row['Nombre de Actividad'], axis=1)

In [16]:
# Cambiar con la segunda columna - Nom_Sector_Activitat
censo_clean['Nombre de Actividad'] = censo_clean.apply(lambda row: row['Nom_Sector_Activitat'] if row['Nombre de Actividad'] in first_lambda else row['Nombre de Actividad'], axis=1)

In [17]:
# pd.set_option('display.max_rows', None)
# censo_clean['Nombre de Actividad'].value_counts()

In [18]:
censo_clean.head(5)

,ID_Bcn_2019,Actividad Economica,Nombre de Actividad,Nom_Sector_Activitat,Nom_Grup_Activitat,Nom_Local,Latitud,Longitud,Nom_Districte,Nombre del Barrio
0,1075454,For Profit,Serveis a les empreses i oficines,Serveis,Altres,SORIGUE,41.346101,2.130166,Sants-Montjuïc,la Marina del Prat Vermell
1,1075453,For Profit,Activitats emmagatzematge,Serveis,Altres,CEJIDOS SIVILA S.A,41.345939,2.129560,Sants-Montjuïc,la Marina del Prat Vermell
2,1075451,For Profit,Serveis a les empreses i oficines,Serveis,Altres,QUALITY ESPRESO,41.345591,2.128543,Sants-Montjuïc,la Marina del Prat Vermell
3,1075449,For Profit,Administració,Altres,Altres,CLD,41.346262,2.130599,Sants-Montjuïc,la Marina del Prat Vermell
4,1075448,For Profit,Activitats de transport,Serveis,Altres,"CATALANA DEL BUTANO,S.A",41.346514,2.131271,Sants-Montjuïc,la Marina del Prat Vermell


In [172]:
# censo_clean.to_csv('../Datos CSV/censo_clean.csv', index=False)

In [22]:
# censo_clean.to_json('../JSON listos/censo_clean.json', orient='records')  

### CSV Centro Comercial

In [130]:
# Load the CSV - Centro Comercial
centros_comerciales = pd.read_csv('../Datos CSV/opendatagrans-centres-comercials.csv', encoding='utf-16')
# Add new column for categorization
centros_comerciales['Tipo_Centro_Comercio'] = 'Centro Comercial'

### Galerias Comerciales

In [132]:
# Load the CSV - Galerias Comerciales
galerias = pd.read_csv('../Datos CSV/opendata_galeries-comercials.csv', encoding='utf-16')
# Add new column for categorization - Galerias Comerciales
galerias['Tipo_Centro_Comercio'] = 'Galeria Comercial'

### Mercats Fires al Carrer

In [134]:
# Add new column for categorization - Galerias Comerciales
mercats_fires = pd.read_csv('../Datos CSV/opendata_mercats-fires-al-carrer.csv', encoding='utf-16')
mercats_fires['Tipo_Centro_Comercio'] = 'Feria'

### Mercados Municipales

In [136]:
mercats_municipals = pd.read_csv('../Datos CSV/opendata_mercats-municipals.csv', encoding='utf-16')
mercats_municipals['Tipo_Centro_Comercio'] = 'Mercado Municipal'

### Concatenation of Centros de Comercio

In [138]:
comercio_bcn = pd.concat([centros_comerciales, galerias, mercats_fires, mercats_municipals])
comercio_cols = ['register_id','name','addresses_road_name','addresses_start_street_number','addresses_district_name','addresses_zip_code','geo_epgs_4326_x','geo_epgs_4326_y','Tipo_Centro_Comercio']
filtered_comercio = pd.DataFrame(comercio_bcn[comercio_cols])
filtered_comercio